In [1]:
import os
import pickle
from typing import Dict, Set, List,TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from pydantic import BaseModel,Field
from langgraph.graph import StateGraph,END,START
from langchain_core.retrievers import BaseRetriever
from langchain_core.messages import HumanMessage
from langchain.retrievers.multi_query import MultiQueryRetriever
from prompt import multiquery_or_ambigious_system_prompt,retrieval_argumented_generation_system_prompt, category_classification_system_prompt,multiquery_splitter_system_prompt

In [ ]:

# ---------------- CONFIG ---------------- #
BASE_DIR = "./collections"     # put subfolders here (each subfolder = one collection)
INDEX_DIR = "./faiss_indexes"  # where per-collection FAISS indexes will be saved
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', api_key="")
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
# ---------------------------------------- #


def discover_collections(base_dir: str) -> List[str]:
    """Return sorted list of subfolder names under base_dir (each becomes a collection)."""
    os.makedirs(base_dir, exist_ok=True)
    names = [
        name for name in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, name)) and not name.startswith(".")
    ]
    return sorted(names)


def get_index_path(collection_name: str) -> str:
    return os.path.join(INDEX_DIR, f"{collection_name}_index")


def get_metadata_path(collection_name: str) -> str:
    return os.path.join(INDEX_DIR, f"{collection_name}_processed.pkl")


def load_processed_files(collection_name: str) -> Set[str]:
    path = get_metadata_path(collection_name)
    if os.path.exists(path):
        with open(path, "rb") as f:
            return pickle.load(f)
    return set()


def save_processed_files(collection_name: str, processed_files: Set[str]):
    path = get_metadata_path(collection_name)
    with open(path, "wb") as f:
        pickle.dump(processed_files, f)


def split_docs(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    return splitter.split_documents(docs)


def load_documents_from_folder(folder_path, file_names):
    """Load documents from the list of file names (supports .txt, .pdf, .docx)."""
    docs = []
    for file in file_names:
        file_path = os.path.join(folder_path, file)
        ext = file.lower().split(".")[-1]

        try:
            if ext == "txt":
                loader = TextLoader(file_path, encoding="utf-8")
            elif ext == "pdf":
                loader = PyPDFLoader(file_path)
            elif ext in ["docx", "doc"]:
                loader = Docx2txtLoader(file_path)
            else:
                print(f"⚠️ Skipping unsupported file type: {file}")
                continue

            docs.extend(loader.load())
        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")
    return docs


def build_or_update_collection(collection_name: str):
    """Create or update FAISS index for a collection folder."""
    folder_path = os.path.join(BASE_DIR, collection_name)
    index_path = get_index_path(collection_name)

    os.makedirs(INDEX_DIR, exist_ok=True)
    os.makedirs(folder_path, exist_ok=True)

    processed_files = load_processed_files(collection_name)

    # detect new files (supported extensions)
    all_files = {
        f for f in os.listdir(folder_path)
        if f.lower().endswith((".txt", ".pdf", ".docx"))
    }
    new_files = all_files - processed_files

    vectordb = None
    if os.path.exists(index_path):
        # load existing FAISS index
        try:
            print(f"Loading existing FAISS index for collection: {collection_name}")
            vectordb = FAISS.load_local(index_path, EMBEDDINGS, allow_dangerous_deserialization=True)
        except Exception as e:
            print(f"⚠️ Failed to load existing index for {collection_name}: {e}. Will try to recreate if there are docs.")

    if new_files:
        print(f"📂 Found new files in {collection_name}: {new_files}")
        new_docs = load_documents_from_folder(folder_path, new_files)
        chunks = split_docs(new_docs)

        if chunks:
            if vectordb:
                vectordb.add_documents(chunks)
            else:
                vectordb = FAISS.from_documents(chunks, EMBEDDINGS)

            try:
                vectordb.save_local(index_path)
            except Exception as e:
                print(f"❌ Error saving index for {collection_name}: {e}")

            processed_files.update(new_files)
            save_processed_files(collection_name, processed_files)
        else:
            print(f"⚠️ No valid text chunks found in new files for {collection_name}")

    if vectordb is None:
        # No index and no new docs -> return None (safe handling by caller)
        print(f"⚠️ No documents for {collection_name}. Skipping index creation.")
        return None

    return vectordb


def query_collection(dbs, collection_name: str, query: str, k=3):
    """Safely query a collection. Returns list of Documents or empty list."""
    db = dbs.get(collection_name)
    if db is None:
        print(f"⚠️ No index found for '{collection_name}'. Add documents to '{os.path.join(BASE_DIR, collection_name)}' and re-run.")
        return []
    return db.similarity_search(query, k=k)


def main(query : str , sample_collection : str):

    # discover collection folders dynamically
    collections = discover_collections(BASE_DIR)

    if not collections:

        return f"ℹ️ No collections found in '{BASE_DIR}'. Create subfolders there (one per collection), then run again."
    else:
        
        print(f"Discovered collections: {collections}")

    dbs: Dict[str, FAISS] = {}

    print("\n🔍 Building/Updating FAISS Indexes...\n")

    for col in collections:

        dbs[col] = build_or_update_collection(col)

    print("\n✅ Summary of collections:")

    for col in collections:

        db = dbs.get(col)

        if db is None:

            print(f" - {col}: 0 documents indexed")

        else:

            try:

                count = db.index.ntotal

            except Exception:

                count = "unknown"

            print(f" - {col}: {count} documents indexed")

    if sample_collection:

        results = query_collection(dbs, sample_collection, query, k=3)

        print(f"......{len(results)} OF CHUNKS RETRIEVED FOR {query}")

        if results:

            for r in results:

                return  f"{r.page_content[:300].replace('\n', ' ')}"
            
        else:

            return f"\n⚠️ No results found for {sample_collection}."




In [ ]:
#SCHEMAS

class MultiQueryOutput(BaseModel):
    multiple_queries: List[str] = Field(description="List of individual queries derived from the input multiquery.")

class collection_classification(BaseModel):
    collection_name : str  = Field(description = f'contain value from one of the defined category which is relatable to query') 

class multi_or_ambigious_classification(BaseModel):
    query_identification : str  = Field(description = f"contain value from one of the following : ['AMBIGIOUS', 'MULTIQUERY', 'NORMAL']") 

class RAG_State(TypedDict):
    input : str
    output : str

llm = ChatOpenAI(model='gpt-4o-mini', api_key="")

class DummyRetriever(BaseRetriever):
    def get_relevant_documents(self, query): return []
    async def aget_relevant_documents(self, query): return []

mqr = MultiQueryRetriever.from_llm(retriever=DummyRetriever(), llm=llm)


C:\Users\admin\AppData\Local\Temp\ipykernel_2516\1174643080.py:18: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class DummyRetriever(BaseRetriever):
C:\Users\admin\AppData\Local\Temp\ipykernel_2516\1174643080.py:18: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class DummyRetriever(BaseRetriever):


In [4]:
#identifying collection to search relevant vector store


def retrival(query : str) :
    print(f"\n\nRETRIEVAL FUNCTION CALLED....\n\n")
    res= discover_collections(BASE_DIR)
    print(res)

    classification = llm.with_structured_output(collection_classification).invoke([HumanMessage(content = category_classification_system_prompt.format(res = res, query = query))])

    print(F'\nCLASSIFIED CATEGORY : {classification.collection_name}\n')

    result = main(query,classification.collection_name)

    print(f"\n🔎 Query Results ({result}):")

    return result

In [5]:

# Function to split queries using LLM
def split_multiquery_llm_and_return_output(query: str) -> List[str]:

    print(f"\n\MULTI QUERY SPLITTER FUNCTION CALLED....\n\n")

    structured_llm = llm.with_structured_output(MultiQueryOutput)

    multiple_query_list = structured_llm.invoke(
        [HumanMessage(content=multiquery_splitter_system_prompt.format(query=query))]
    )
    print(f"\n\nMULTI QUERY SPLITTED INTO -({multiple_query_list})\n\n")

    list_output = []
    for q in multiple_query_list.multiple_queries:
        print(f"\n\nsearching for query......'{q}'")
        output = retrival(q)
        list_output.append(output)
                
    print(f"\n\nMULTI QUERY OUTPUT -({list_output})\n\n")

    return list_output

<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\admin\AppData\Local\Temp\ipykernel_2516\1824605591.py:4: SyntaxWarning: invalid escape sequence '\M'
  print(f"\n\MULTI QUERY SPLITTER FUNCTION CALLED....\n\n")


In [6]:
#classifying into MULTIQUERY, AMBIGUOUS, NORMAL


def retrieval_for_multiquery_ambigious_or_normal(query : str) -> List:

    print(f"\n\retrieval_for_multiquery_ambigious_or_normal FUNCTION CALLED....\n\n")

    multiquery_ambigious_classification = llm.with_structured_output(multi_or_ambigious_classification).invoke([HumanMessage(content = multiquery_or_ambigious_system_prompt.format(query = query))])

    if multiquery_ambigious_classification.query_identification == 'MULTIQUERY':
        print('MULTIQUERY')

        multi_query_result = split_multiquery_llm_and_return_output(query)

        return multi_query_result

    elif multiquery_ambigious_classification.query_identification == 'AMBIGIOUS':

        print('AMBIGIOUS')

        rewritten_queries = mqr.llm_chain.invoke({"question": query})

        print(f"QUERIES GEERATED FOR AMBIGIOUS QUERY : {rewritten_queries}")

        ambiguous_retrieval_list =[]

        for query in rewritten_queries:
            
            ambiguous_retrieval_list.append(retrival(query))

        return ambiguous_retrieval_list

    elif multiquery_ambigious_classification.query_identification == 'NORMAL':

        result = retrival(query)

        return result

    else :
        return ['no data available']

In [10]:
def rag_chain(state : RAG_State) -> RAG_State:

    data = retrieval_for_multiquery_ambigious_or_normal(state['input'])

    context = ', '.join(data)

    print(f"FINAL CONTEXT : {context}")

    rag_result = llm.invoke([HumanMessage(retrieval_argumented_generation_system_prompt.format(query = state['input'], context = context))])

    print('LLM RESULT : ',rag_result.content)

    return {'output' : rag_result.content}


In [11]:
rag_graph = StateGraph(RAG_State)
rag_graph.add_node('rag_retrieval',rag_chain)
rag_graph.add_edge(START,'rag_retrieval')
rag_graph.add_edge('rag_retrieval',END)
ready_rag_graph = rag_graph.compile()

In [12]:
query = {"input":'what is frontend developmeeeement is','output':None}

output = ready_rag_graph.invoke(query)

print(output['output'])


etrieval_for_multiquery_ambigious_or_normal FUNCTION CALLED....


AMBIGIOUS
QUERIES GEERATED FOR AMBIGIOUS QUERY : ['What does frontend development entail?', 'Can you explain the key aspects of frontend development?', 'What are the main components involved in frontend development?']


RETRIEVAL FUNCTION CALLED....


['abhiraj', 'agnetic_ai', 'frontend_dovelopment', 'machine_learning']

CLASSIFIED CATEGORY : frontend_dovelopment

Discovered collections: ['abhiraj', 'agnetic_ai', 'frontend_dovelopment', 'machine_learning']

🔍 Building/Updating FAISS Indexes...

Loading existing FAISS index for collection: abhiraj
⚠️ No documents for agnetic_ai. Skipping index creation.
Loading existing FAISS index for collection: frontend_dovelopment
Loading existing FAISS index for collection: machine_learning

✅ Summary of collections:
 - abhiraj: 1 documents indexed
 - agnetic_ai: 0 documents indexed
 - frontend_dovelopment: 6 documents indexed
 - machine_learning: 9 documents indexed
......3 OF CHUN